# This notebook is used to remove asphalt from the Natura 2000 region of Coepelduynen.
Weirdly enough parking spaces and some roads are still included in the official polygon region of Coepelduynen.
We made the decision to remove these ashpalt regions from the satellite images.

We combine here model results from a previous model which predicted asphalt supplement with handdrawn annotations.

In [1]:
%matplotlib notebook
import glob
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from satellite_images_nso._manipulation import nso_manipulator

c:\ProgramData\Anaconda3\lib\site-packages\geopandas\_compat.py:115: UserWarning: The Shapely GEOS version (3.4.3-CAPI-1.8.3 r4285) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [ ]:
# Note this .shp file is the result of a model which predicted Asphalt but was not fully correct
gdf = gpd.read_file("E:/output/Coepelduynen_segmentations/20230908_110020_PNEO-04_1_1_30cm_RD_12bit_RGBNED_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_re_ndvi.shp")
gdf = gdf[gdf['label'] == "Asphalt"]
gdf = gdf.explode()
gdf['area'] = gdf['geometry'].area
gdf['perimeter'] = gdf['geometry'].length

In [ ]:
# Filter out the faulty ashpalt regions based on bad model preformance.
gdf[(gdf["area"] > 50) & (gdf['perimeter'] > 150) ].unary_union

In [ ]:
# Create a new GeoDataFrame with the resulting MultiPolygon
gdf_save = gpd.GeoDataFrame({'geometry': [gdf[(gdf["area"] > 50) & (gdf['perimeter'] > 150) ].unary_union]}, geometry='geometry', crs=gdf.crs)

In [ ]:
gdf_save.to_file("C:/repos/satellite-images-nso-datascience/data/annotations/Coepelduynen/asfalt_detection_coepelduynen.shp")

### Now merge with handdrawn annotated data

In [ ]:
annotated_gdf = gpd.read_file("C:/repos/satellite-images-nso-datascience/data/annotations/Coepelduynen/asfalt_annotated_coepelduynen.geojson")
annotated_gdf['geometry'] = annotated_gdf['geometry'].buffer(0)

In [ ]:
# Check the result
gdf_save.append(annotated_gdf,ignore_index=True).drop("id",axis=1).unary_union

In [ ]:
gdf_save = gpd.GeoDataFrame({'geometry': [gdf_save.append(annotated_gdf,ignore_index=True).drop("id",axis=1).unary_union]}, geometry='geometry', crs=gdf.crs)
gdf_save.to_file("C:/repos/satellite-images-nso-datascience/data/annotations/Coepelduynen/asfalt_coepelduynen.shp")

### Extract from the original file

In [ ]:
coepelduynen_gdf = gpd.read_file("C:/Users/pzhadmin/Documents/natura2000_coepelduynen.geojson")
coepelduynen_gdf =coepelduynen_gdf.to_crs("EPSG:28992")
diff_gdf = coepelduynen_gdf.difference(gdf_save)
diff_gdf.to_file("C:/Users/pzhadmin/Documents/natura2000_coepelduynen_no_asphalt.geojson")

### Implement on all files

In [ ]:
for file in glob.glob("E:/data/coepelduynen/*ndvi*.tif"):
    file = file.replace("\\","/")
    print(file)
    print("---")
   
    print( file.replace(".tif","_asphalt_crop.tif"))
    
    nso_manipulator.__make_the_crop("C:/Users/pzhadmin/Documents/natura2000_coepelduynen_no_asphalt.geojson",file,file.replace(".tif","_asphalt_crop.tif"), False)